In [20]:
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown widget for environment selection
interface_dropdown = widgets.Dropdown(
    options=["LAWSON-0005", "LAWSON-0006", "LAWSON-0007"],
    value="LAWSON-0006",
    description="Interface:",
)

# Display the widget
# display(widgets.HBox([interface_dropdown, outputdirectory]))
display(widgets.HBox([interface_dropdown]))

### Read Configuration Data

In [21]:
from sqlalchemy import create_engine
import pandas as pd
# Database connection parameters
db_user = 'root'
db_password = 'strong_password'
db_host = 'localhost'
db_name = 'yourdb'
db_port = 33062
interface_id = interface_dropdown.value

# Create a database connection
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
print(interface_id)

LAWSON-0006


In [22]:
# Query to read data for a specific interfaceID
queryInterface = f"SELECT * FROM interface WHERE interface_id = '{interface_id}'"

# Load data into dataframe
dfInterface = pd.read_sql(queryInterface, engine)

mappingId = dfInterface['mapping_id'].astype(str).to_string(index=False)

# Query to read data for a specific interfaceID
queryMapping = f"SELECT * FROM mapping WHERE mapping_id = '{mappingId}'"

# Load data into dataframe
dfMapping = pd.read_sql(queryMapping, engine)

display(dfInterface)
display(dfMapping)

,interface_id,source_system,source_type,source_path,source_delimiter,destination_system,destination_type,destination_path,program_name,mapping_id,interface_description
0,LAWSON-0006,FILE,DELIMITED,None,",",Database,AccountDestinationTableB,None,FileToTable,MAPPING-0002,CSV File- Comma DELIMITED Testing Concept


,mapping_id,source_field,start_position,source_length,destination_field,destination_data_type,destination_length,mapping_function
0,MAPPING-0002,ID,0,4,ID,INT,NaN,PASSTHROUGH
1,MAPPING-0002,Name,4,14,Name,VARCHAR,50.0,PASSTHROUGH
2,MAPPING-0002,Date,18,8,Date,DATE,NaN,PASSTHROUGH
3,MAPPING-0002,Amount,26,8,Amount,FLOAT,NaN,PASSTHROUGH


#### Read Fixed Length File

In [23]:
import pandas as pd

#LOGIC FOR FIXED WIDTH FILE
if(dfInterface['source_type'].head(1).to_string(index=False).upper()):
    colspecs = [(row['start_position'], row['start_position'] + row['source_length']) for index, row in dfMapping.iterrows() if row['source_field'] is not None]
    names = [row['source_field'] for index, row in dfMapping.iterrows() if row['source_field'] is not None]  # Column names
    dfSource = pd.read_fwf('SourceFileFixedWidth.dat', colspecs=colspecs, names=names, header=None)

#LOGIC FOR DELIMITED FILE
elif(dfInterface['source_type'].to_string(index=False).upper() == 'DELIMITED'):
    delimiter = dfMapping['source_delimiter'].to_string(index=False)
    names = [row['source_field'] for index, row in dfMapping.iterrows() if row['source_field'] is not None]  # Column names
    dfSource = pd.read_csv('Delimited.dat', delimiter=delimiter, names=names, header=None)
#dfSource = pd.read_csv('SourceFileDelimited.dat', delimiter='|', names=names, header=None)

print(dfSource)

     ID         Name      Date   Amount
0  1234     John Doe  20231231   100.00
1  5678   Jane Smith  20240115   200.50
2  9012  Peter Jones  20240229    50.25
3  3456   Mary Brown  20240315  4000.75


#### Map to Destination

In [24]:
# Create an empty DataFrame with the same columns as dfSource
dfDestination = pd.DataFrame(columns=dfSource.columns)
#Solidify the Mapping logic and data transformation
    

destinationFields = ''
for index, row in dfMapping.iterrows():
    destination = row['destination_field']
    source = row['source_field']
    mapping_function = row['mapping_function']
    destination_type = row['destination_data_type']
    if mapping_function == 'PASSTHROUGH':
        if index == len(dfMapping) - 1:
            destinationFields += destination
        else:
            destinationFields += destination + ','
    else:
        if index == len(dfMapping) - 1:
            destinationFields += f'{mapping_function} AS {destination}'
        else:
            destinationFields += f'{mapping_function} AS {destination},'

destinationQuery = f"SELECT {destinationFields} FROM dfSource"
print(destinationQuery)
print(destinationFields)

SELECT ID,Name,Date,Amount FROM dfSource
ID,Name,Date,Amount


In [25]:
import pandasql as psql

dfDestination = psql.sqldf(destinationQuery, globals())

print(dfDestination)

     ID         Name      Date   Amount
0  1234     John Doe  20231231   100.00
1  5678   Jane Smith  20240115   200.50
2  9012  Peter Jones  20240229    50.25
3  3456   Mary Brown  20240315  4000.75


#### Load into Destination

In [26]:
# Extract the destination table name from the interface table
destination_table = dfInterface['destination_type'].iloc[0]
print(destination_table)
# Load the dataframe into the SQL table
dfDestination.to_sql(destination_table, engine, if_exists='replace', index=False)

print(f"Data loaded into table {destination_table} successfully.")

AccountDestinationTableB
Data loaded into table AccountDestinationTableB successfully.
